
    # Transformer Fundamentals – Guided Notebook 02 — Attention Mechanism (Q/K/V)
    **Date:** 2025-10-29  
    **Style:** Guided, hands-on; from-scratch first, then frameworks; interactive visuals

    ## Learning Objectives

- Compute Q, K, V projections from token embeddings.
- Build scaled dot-product attention from scratch.
- Inspect shapes and parameters for single-head vs multi-head attention.
- Compare a NumPy implementation to PyTorch’s nn.MultiheadAttention.


    ## TL;DR
    Q queries, K keys, V values: attention selects information by weighting V according to Q·K similarity.


## Concept Overview
From embeddings `X` (shape `[T, d_model]`), linear layers produce `Q = XW_Q`, `K = XW_K`, `V = XW_V`.
Attention weights are computed via scaled dot product.


In [ ]:

# %% [setup] Environment check & minimal installs (run once per kernel)
# Target: Python 3.12.12, PyTorch 2.5+, transformers 4.44+, datasets 3+, ipywidgets 8+, matplotlib 3.8+
import sys, platform, subprocess, os

print("Python:", sys.version)
print("Platform:", platform.platform())

# Optional: uncomment to install/upgrade on this machine (internet required)
# !pip install --upgrade pip
# !pip install "torch>=2.5" "transformers>=4.44" "datasets>=3.0.0" "ipywidgets>=8.1.0" "matplotlib>=3.8" "umap-learn>=0.5.6"

try:
    import torch
    print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA device name:", torch.cuda.get_device_name(0))
except Exception as e:
    print("PyTorch not available yet:", e)

%config InlineBackend.figure_format = 'retina'
from IPython.display import display, HTML
try:
    import ipywidgets as widgets
    from ipywidgets import interact, interactive
    print("ipywidgets:", widgets.__version__)
except Exception as e:
    print("ipywidgets not available yet:", e)

import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)


In [ ]:

# %% [utils] Small helpers used throughout
import numpy as np

def softmax(x, axis=-1):
    x = x - np.max(x, axis=axis, keepdims=True)
    e = np.exp(x)
    return e / np.sum(e, axis=axis, keepdims=True)

def cosine_sim(a, b, eps=1e-9):
    a_norm = a / (np.linalg.norm(a, axis=-1, keepdims=True) + eps)
    b_norm = b / (np.linalg.norm(b, axis=-1, keepdims=True) + eps)
    return np.dot(a_norm, b_norm.T)

def show_heatmap(mat, xticklabels=None, yticklabels=None, title=""):
    plt.figure()
    plt.imshow(mat, aspect="auto")
    plt.colorbar()
    if xticklabels is not None: plt.xticks(range(len(xticklabels)), xticklabels, rotation=45, ha="right")
    if yticklabels is not None: plt.yticks(range(len(yticklabels)), yticklabels)
    plt.title(title)
    plt.tight_layout()
    plt.show()


In [ ]:

# %% [from-scratch] Single-head attention (NumPy)
T, d_model, d_k = 6, 16, 16
X = np.random.randn(T, d_model) * 0.2
W_Q = np.random.randn(d_model, d_k) * 0.1
W_K = np.random.randn(d_model, d_k) * 0.1
W_V = np.random.randn(d_model, d_k) * 0.1

Q = X @ W_Q
K = X @ W_K
V = X @ W_V

scores = (Q @ K.T) / np.sqrt(d_k)   # [T, T]
attn = softmax(scores, axis=-1)
out = attn @ V                      # [T, d_k]

print("scores shape:", scores.shape, "attn shape:", attn.shape, "out shape:", out.shape)
show_heatmap(attn, title="Attention Weights (Toy)")


In [ ]:

# %% [framework] PyTorch MultiheadAttention check
import torch, torch.nn as nn

torch.manual_seed(42)
T, d_model, nheads = 6, 32, 4
mha = nn.MultiheadAttention(embed_dim=d_model, num_heads=nheads, batch_first=True)
x = torch.randn(1, T, d_model)
out, w = mha(x, x, x, need_weights=True)
print("out:", out.shape, "weights:", w.shape)  # weights: [1, heads, T, T]



---
### Bonus: Multilingual Extension
- Swap the tokenizer/model for a multilingual variant (e.g., `bert-base-multilingual-cased` or `xlm-roberta-base`).
- Repeat a small slice of the notebook (tokenization, attention map) on non-English sentences and compare.



---
## Reflection & Next Steps
- What changed when you tweaked dimensions, temperatures, or prompts?
- Where did the attention concentrate, and did it match your intuition?
- Re-run the interactive widgets on your own text.
- Save a copy of the figures that best illustrate your understanding.
